In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import re #regukar expression
import warnings
import pytz #date-time conversion
import requests #to get info from server
import time
import datetime as dt
import json

from pandas.io.json import json_normalize
from IPython.display import display
from datetime import datetime
from pandas.io.json import json_normalize
from datetime import date, timedelta
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  #ignores the warning message and code runs faster # default='warn'

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.options.display.float_format = '{:.2f}'.format

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.options.display.float_format = '{:.2f}'.format

from IPython.display import display
from IPython.display import HTML
import geopy.distance
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button style="color:"black";right:75%;" onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

<br>

# Weather Source Full Documentation :
### https://developer.weathersource.com/documentation/resources/

<br>

## **Get Points History**

Description : Returns a collection of weather history data for a latitude/longitude point.

Our patented OnPoint™ weather data is not confined to weather station locations, but is computed by overlaying multiple past weather sources (including weather station, satellite, radar, weather balloon, and model data – among others). The resulting quality-checked, error-corrected, gap-filled, and contiguous data is uniquely derived for possible every latitude/longitude location. This resource is globally available, everywhere there is landmass (except Antartica), for 2007 onward.

Requires a subscription plan with the “read_history” and “access_points” capabilities to access this resource.

Resource URL : https://api.weathersource.com/v1/{{api_key}}/points/{{latitude}},{{longitude}}/history.{{format}}

Documentation Used : https://developer.weathersource.com/documentation/resources/get-points-latitude_and_longitude-history/

In [ ]:
def ws_historical_data(start, end, lat, long, period='day', fields='all'):
    '''
    If duration is more than 1 year separate calls should be used
    Timestamp should be converted to ISO 8601 format
    Docstring with examples and function return values:
    
    Input :
    start - (%Y-%m-%d) format
    end - (%Y-%m-%d) format
    lat - latitude 
    long - longitude
    period - hour, day (default=day)
    
    Output : return a callection of weather historical data for a latitude/longitude point
    
    '''
    key = 'e721181f854ac2268ee8'
    start = pd.to_datetime(start,format='%Y-%m-%d')
    end = pd.to_datetime(end, format='%Y-%m-%d')
    
    start = start.strftime('%Y-%m-%dT%H:%M:%S')
    end = end.strftime('%Y-%m-%dT%H:%M:%S')
    
    weather_ = pd.DataFrame()
    link = 'https://api.weathersource.com/v1/'+key+'/points/'+lat+','+long+'/history.json?period='+period+'&timestamp_between='+start+','+end+'&fields='+fields
    print(link)
    response = requests.get(link)
    json_obj = json.loads(response.content.decode('utf-8'))
    weather_ = json_normalize(json_obj)
    
    return weather_

In [ ]:
start_time = "2020-06-09"
end_time = "2020-06-15"
location = ['39.9613', '-86.4034']

data = ws_historical_data(start=start_time, end=end_time, lat=location[0], long=location[1], period='hour')
data.head()

In [ ]:
start_time = "2020-06-09"
end_time = "2020-06-15"
location = ('39.9613', '-86.4034')

data = ws_historical_data(start=start_time, end=end_time, lat=location[0], long=location[1], period='day')
data.head()

## **Get Points Forecast**

Description : Returns up to 15 days of daily weather forecast data or 240 hours of hourly weather forecast data for a latitude/longitude point.

Our patented OnPoint™ system uniquely derives contiguous data for every possible latitude/longitude location. This resource is globally available, everywhere there is landmass (except Antartica).

Requires a subscription plan with the “read_forecast” and “access_points” capabilities to access this resource.

Resource Url : https://api.weathersource.com/v1/{{api_key}}/points/{{latitude}},{{longitude}}/forecast.{{format}}

Documentation Used : https://developer.weathersource.com/documentation/resources/get-points-latitude_and_longitude-forecast/

In [ ]:
def ws_forecast_data(start, end, lat, long, period='day', fields='all'):
    '''
    Timestamp should be converted to ISO 8601 format
    Docstring with examples and function return values:
    
    Input :
    start - (%Y-%m-%d) format
    end - (%Y-%m-%d) format
    lat - latitude 
    long - longitude
    period - hour, day (default=day)
    
    Output : returns forecast data upto 15 days ahead of forecast data and 240 hours of hourly weather data for a latitude/longitude point
    
    '''
    key = 'e721181f854ac2268ee8'
    start = pd.to_datetime(start,format='%Y-%m-%d')
    end = pd.to_datetime(end, format='%Y-%m-%d')
    
    start = start.strftime('%Y-%m-%dT%H:%M:%S')
    end = end.strftime('%Y-%m-%dT%H:%M:%S')
    
    weather_ = pd.DataFrame()
    link = 'https://api.weathersource.com/v1/'+key+'/points/'+lat+','+long+'/forecast.json?period='+period+'&timestamp_between='+start+','+end+'&fields='+fields
    response = requests.get(link)
    json_obj = json.loads(response.content.decode('utf-8'))
    weather_ = json_normalize(json_obj)
    
    return weather_

In [ ]:
start_time = "2020-06-23"
end_time = "2020-06-30"
location = ('39.9613', '-86.4034')

data = ws_forecast_data(start=start_time, end=end_time, lat=location[0], long=location[1], period='day')
data.head()

In [ ]:
start_time = "2020-06-23"
end_time = "2020-06-30"
location = ('39.9613', '-86.4034')

data = ws_forecast_data(start=start_time, end=end_time, lat=location[0], long=location[1], period='hour')
data.head()

#  Dark - sky : https://darksky.net/dev/docs

## **Get Daily Data**

In [ ]:
def getdarkskydatadaily(start,end,loc_name):
    """
    
    Input - start_date, end_date, API KEY from https://darksky.net/dev/register (now closed APPLE has acqired darksky)
    Output -daily level weather data from start date to end date for specified location
    
    """
    #api_key = '12ea26d5539f1add37b448b7db20f6b8'
    api_key ='03f7d7221972645470c7e8c6f35edd85'
    headers = {'User-Agent': 'Chrome/78and.0.3865.90'}
    http_proxy  = "http://10.245.5.249:8080"
    https_proxy = "https://10.245.5.249:8080"
    ftp_proxy   = "ftp://10.245.5.249:8080"

    proxyDict = { 
                "http"  : http_proxy, 
                "https" : https_proxy, 
                "ftp"   : ftp_proxy
                 }
    
    weather_data_daily = pd.DataFrame()
    date_list = []
    
    # get month,year and day from start date
    start_date = dt.datetime.strptime(start, "%Y-%m-%d")
    start_month = start_date.month
    start_year = start_date.year
    start_day = start_date.day
    
    # get month,year and day from end date
    end_date = dt.datetime.strptime(end, "%Y-%m-%d")
    end_month = end_date.month
    end_year = end_date.year
    end_day = end_date.day
    
    # store all date from start date to end date in date list 
    start_date = date(start_year, start_month, start_day)
    end_date = date(end_year, end_month, end_day)
    delta = timedelta(days=1)
    
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += delta
    
    # latitude of the location markers
    sites_latitude = {
        'Marker 1' : '39.9613','Marker 2' : '39.8971','Marker 3' : '39.9060','Marker 4' : '39.9024','Marker 5' : '39.8960','Marker 6' : '39.8339',
        'Marker 7' : '39.8412','Marker 8' : '39.8381','Marker 9' : '39.8386','Marker 10' : '39.7579','Marker 11' : '39.7621','Marker 12' : '39.7621',
        'Marker 13' : '39.7695','Marker 14' : '39.6617','Marker 15' : '39.6639','Marker 16' : '39.6702','Marker 17' : '39.6744','Marker 18' : '39.5909',
        'Marker 19' : '39.5295','Marker 20' : '39.5475'
    }
    
    # longitude of the location markers
    sites_longitude = {
        'Marker 1' : '-86.4034','Marker 2' : '-86.3045','Marker 3' : '-86.2001','Marker 4' : '-86.0738','Marker 5' : '-85.9783','Marker 6' : '-86.3155',
        'Marker 7' : '-86.2056','Marker 8' : '-86.0985','Marker 9' : '-85.9811','Marker 10' : '-86.3155','Marker 11' : '-86.2042','Marker 12' : '-86.0923',
        'Marker 13' : '-85.9708','Marker 14' : '-86.2935','Marker 15' : '-86.1823','Marker 16' : '-86.0669','Marker 17' : '-85.9557','Marker 18' : '-86.4212',
        'Marker 19' : '-86.5874','Marker 20' : '-86.2743'
    }
    
    value1 = sites_latitude.get(loc_name)
    value2 = sites_longitude.get(loc_name)
    if(value1 == None and value2 == None):
        print('Unknown Location Name Used: %s',loc_name)
        print('\n')
        print('Use some other Consumer Location or Generation Location Name.. .. ..')
        print('\n')
    else :
        for i in range(len(date_list)):
            #requesting web page
            date_format = '%Y-%m-%dT%H:%M:%S'
            dates = dt.datetime.strptime(date_list[i], '%Y-%m-%d').strftime(date_format)
            links = 'https://api.darksky.net/forecast/'+api_key+'/'+sites_latitude.get(loc_name)+','+sites_longitude.get(loc_name)+','+str(dates)+'?exclude=currently,hourly,alerts?units=us'
            print(links)
            response = requests.get(links,headers=headers,proxies=proxyDict)

            # converting into json
            weather = json.loads(response.content.decode('utf-8'))
            # getting it into data frame
            weather_data = json_normalize(weather['daily']['data'])
            weather_data_daily = weather_data_daily.append(weather_data,sort=True)
    
    weather_data_daily.reset_index(drop=True, inplace=True)
    
    return weather_data_daily

## **Get Hourly Data**

In [ ]:
def getdarkskydatahourly(start,end,loc_name):
    """
    
    Input - start_date, end_date, API KEY from https://darksky.net/dev/register (now closed APPLE has acqired darksky)
    Output - hourly level weather data from start date to end date for specified location
    
    """
    api_key = '12ea26d5539f1add37b448b7db20f6b8'
    
    headers = {'User-Agent': 'Chrome/78and.0.3865.90'}
    http_proxy  = "http://10.245.5.249:8080"
    https_proxy = "https://10.245.5.249:8080"
    ftp_proxy   = "ftp://10.245.5.249:8080"

    proxyDict = { 
                "http"  : http_proxy, 
                "https" : https_proxy, 
                "ftp"   : ftp_proxy
                 }
    
    weather_data_hourly = pd.DataFrame()
    date_list = []
    
    # get month,year and day from start date
    start_date = dt.datetime.strptime(start, "%Y-%m-%d")
    start_month = start_date.month
    start_year = start_date.year
    start_day = start_date.day
    
    # get month,year and day from end date
    end_date = dt.datetime.strptime(end, "%Y-%m-%d")
    end_month = end_date.month
    end_year = end_date.year
    end_day = end_date.day
    
    # store all date from start date to end date in date list 
    start_date = date(start_year, start_month, start_day)
    end_date = date(end_year, end_month, end_day)
    delta = timedelta(days=1)
    
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += delta
    
    # latitude of the location markers
    sites_latitude = {
        'Marker 1' : '39.9613','Marker 2' : '39.8971','Marker 3' : '39.9060','Marker 4' : '39.9024','Marker 5' : '39.8960','Marker 6' : '39.8339',
        'Marker 7' : '39.8412','Marker 8' : '39.8381','Marker 9' : '39.8386','Marker 10' : '39.7579','Marker 11' : '39.7621','Marker 12' : '39.7621',
        'Marker 13' : '39.7695','Marker 14' : '39.6617','Marker 15' : '39.6639','Marker 16' : '39.6702','Marker 17' : '39.6744','Marker 18' : '39.5909',
        'Marker 19' : '39.5295','Marker 20' : '39.5475'
    }
    
    # longitude of the location markers
    sites_longitude = {
        'Marker 1' : '-86.4034','Marker 2' : '-86.3045','Marker 3' : '-86.2001','Marker 4' : '-86.0738','Marker 5' : '-85.9783','Marker 6' : '-86.3155',
        'Marker 7' : '-86.2056','Marker 8' : '-86.0985','Marker 9' : '-85.9811','Marker 10' : '-86.3155','Marker 11' : '-86.2042','Marker 12' : '-86.0923',
        'Marker 13' : '-85.9708','Marker 14' : '-86.2935','Marker 15' : '-86.1823','Marker 16' : '-86.0669','Marker 17' : '-85.9557','Marker 18' : '-86.4212',
        'Marker 19' : '-86.5874','Marker 20' : '-86.2743'
    }
    
    value1 = sites_latitude.get(loc_name)
    value2 = sites_longitude.get(loc_name)
    if(value1 == None and value2 == None):
        print('Unknown Location Name Used: %s',loc_name)
        print('\n')
        print('Use some other Consumer Location or Generation Location Name.. .. ..')
        print('\n')
    else :
        for i in range(len(date_list)):
            #requesting web page
            date_format = '%Y-%m-%dT%H:%M:%S'
            dates = dt.datetime.strptime(date_list[i], '%Y-%m-%d').strftime(date_format)
            links = 'https://api.darksky.net/forecast/'+api_key+'/'+sites_latitude.get(loc_name)+','+sites_longitude.get(loc_name)+','+str(dates)+'?exclude=currently,daily,alerts?units=us'
            print(links)
            response = requests.get(links,headers=headers,proxies=proxyDict)

            # converting into json
            weather = json.loads(response.content.decode('utf-8'))
            # getting it into data frame
            weather_data = json_normalize(weather['hourly']['data'])
            weather_data_hourly = weather_data_hourly.append(weather_data,sort=True)
    
    weather_data_hourly.reset_index(drop=True, inplace=True)
    
    return weather_data_hourly

## **CimaCell Station Requests**

In [ ]:
def actual_station_pull_request(start, end, loc_name):
    '''
    
    Input
    start - start_date for pull request in string format
    end - end_date for pull requqest in string format
    loc_name - name of the location in string format
    
    Output
    actual weather data from station pull requqest in raw format from climacell (20 minutes interval)
    
    '''
    headers = {'User-Agent': 'Chrome/78and.0.3865.90'}
    http_proxy  = "http://10.245.5.249:8080"
    https_proxy = "https://10.245.5.249:8080"
    ftp_proxy   = "ftp://10.245.5.249:8080"

    proxyDict = { "http"  : http_proxy, "https" : https_proxy, "ftp"   : ftp_proxy }
    
    # latitude of the location markers
    sites_latitude = {
        'Marker 1' : '39.9613','Marker 2' : '39.8971','Marker 3' : '39.9060','Marker 4' : '39.9024','Marker 5' : '39.8960','Marker 6' : '39.8339',
        'Marker 7' : '39.8412','Marker 8' : '39.8381','Marker 9' : '39.8386','Marker 10' : '39.7579','Marker 11' : '39.7621','Marker 12' : '39.7621',
        'Marker 13' : '39.7695','Marker 14' : '39.6617','Marker 15' : '39.6639','Marker 16' : '39.6702','Marker 17' : '39.6744','Marker 18' : '39.5909',
        'Marker 19' : '39.5295','Marker 20' : '39.5475'
    }
    
    # longitude of the location markers
    sites_longitude = {
        'Marker 1' : '-86.4034','Marker 2' : '-86.3045','Marker 3' : '-86.2001','Marker 4' : '-86.0738','Marker 5' : '-85.9783','Marker 6' : '-86.3155',
        'Marker 7' : '-86.2056','Marker 8' : '-86.0985','Marker 9' : '-85.9811','Marker 10' : '-86.3155','Marker 11' : '-86.2042','Marker 12' : '-86.0923',
        'Marker 13' : '-85.9708','Marker 14' : '-86.2935','Marker 15' : '-86.1823','Marker 16' : '-86.0669','Marker 17' : '-85.9557','Marker 18' : '-86.4212',
        'Marker 19' : '-86.5874','Marker 20' : '-86.2743'
    }
    
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    
    check_end_date = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))
    check_start_date = check_end_date - timedelta(days=28)
    
    if(start < check_start_date):
        print("Bad start date, start date should not be less than 28 days prior to current date")
    elif(end > check_end_date):
        print("Station endpoint requqest will pull data for previous 15 days only")
    else :
        print("We are good to go")
    
    date_list = []
    delta = timedelta(days=1)

    while start <= end:
        date_list.append(start.strftime("%Y-%m-%d"))
        start += delta
    
    weather_data_daily = pd.DataFrame()
    value1 = sites_latitude.get(loc_name)
    value2 = sites_longitude.get(loc_name)
    
    if(value1 == None and value2 == None):
        print('Unknown Location Name Used: %s',loc_name)
        print('\n')
        print('Use some location name which already defined in sites_longitude, sites_latitude dictionary .. .. ..')
        print('\n')
    else :
        for i in range(len(date_list)-1):
            date_format = '%Y-%m-%dT%H:%M:%S'
            start_dates = dt.datetime.strptime(date_list[i], '%Y-%m-%d').strftime(date_format)
            end_dates = dt.datetime.strptime(date_list[i+1], '%Y-%m-%d').strftime(date_format)
        
            url = "https://api.climacell.co/v3/weather/historical/station"
            print(url)
            querystring = {
                "lat": sites_latitude.get(loc_name),"lon": sites_longitude.get(loc_name),
                "unit_system": "us" , "start_time": start_dates, "end_time": end_dates,
                "apikey": "8S1mZisNIgMuHAO4VhGx76Uy4vgblWO8", 
                "fields": {'temp', 'feels_like', 'dewpoint', 'humidity','wind_speed', 'wind_direction', 'wind_gust',
                                  'baro_pressure', 'precipitation', 'precipitation_type','visibility', 'cloud_cover', 'cloud_base', 'cloud_ceiling'}
                          }

            response = requests.request("GET", url, params=querystring, headers=headers,proxies=proxyDict)

            weather_data_hourly = json.loads(response.content.decode('utf-8'))
            weather_data = pd.io.json.json_normalize(weather_data_hourly)
            weather_data_daily = weather_data_daily.append(weather_data, sort=True, ignore_index=True)
            weather_data_daily.reset_index(drop=True, inplace=True)
        
    return weather_data_daily

# **Darksky vs Weather Source (Comparison Plots)**

## **Actual Daily Data**

In [ ]:
start_date = "2020-06-01"
end_date = "2020-06-20"
location = ('39.9613', '-86.4034')
location_marker = 'Marker 1'

weathersource_data = ws_historical_data(start=start_date, end=end_date, lat=location[0], long=location[1], period='day')
darksky_data = getdarkskydatadaily(start_date,end_date,location_marker)
climacell_data = actual_station_pull_request(start_date, end_date, location_marker)

In [ ]:
climacell_data['cloud_cover.value'] = climacell_data['cloud_cover.value'].fillna(0)

climacell_data['observation_time.value'] = pd.to_datetime(climacell_data['observation_time.value'],errors='coerce')
climacell_data['Date'] = climacell_data['observation_time.value'].dt.date

cm_data = climacell_data.groupby(['Date'],as_index=False).agg({'baro_pressure.value' : 'mean', 'dewpoint.value' : 'mean', 'feels_like.value': 'mean', 
                                                               'cloud_cover.value' : 'mean', 'humidity.value' : 'mean', 'temp.value' : 'mean',
                                                               'visibility.value' : 'mean','wind_direction.value' : 'mean', 'wind_speed.value' : 'mean'})

########################################################################################################################################################################################################################
########################################################################################################################################################################################################################

weathersource_data['date_time'] = pd.to_datetime(weathersource_data['timestamp'])
weathersource_data['Date'] = weathersource_data['date_time'].dt.date

########################################################################################################################################################################################################################
########################################################################################################################################################################################################################

darksky_data['date_time'] = pd.to_datetime(darksky_data['time'],unit='s')
darksky_data['date_time']=darksky_data['date_time'].astype(str)
 
#shift data from gmt to est time
def date_convert(date):
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    my_timestamp = date # some timestamp
    old_timezone = pytz.timezone("GMT")
    new_timezone = pytz.timezone("US/Eastern")

    # returns datetime in the new timezone
    my_timestamp_in_new_timezone = old_timezone.localize(my_timestamp).astimezone(new_timezone)
    return pd.to_datetime(my_timestamp_in_new_timezone)
 
darksky_data['date_time'] = darksky_data.apply(lambda x: date_convert(x['date_time']),axis=1)
darksky_data['Date'] = darksky_data['date_time'].dt.date


In [ ]:
print('\n')
print('\n')
display(weathersource_data.head())
print('\n')
print('\n')
print('\n')
print('\n')
display(darksky_data.head())

In [ ]:
weathersource_data.drop(['date_time'],axis=1,inplace=True)
darksky_data.drop(['date_time'],axis=1,inplace=True)

In [ ]:
merged_data = pd.merge(weathersource_data,darksky_data, on=['Date'], how='left')
merged_data.head()

In [ ]:
wind_speed = merged_data.filter(regex=("windS"))
print(list(wind_speed.columns))

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date', 'windSpdAvg', 'windSpdMax', 'windSpdMin', 'windSpeed']], ['Date']))
    new_labels = ['Weather_Source_windSpdAvg', 'Weather_Source_windSpdMax', 'Weather_Source_windSpdMin', 'Darksky_windSpeed']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Speed in Miles per hour',title="Wind Speed Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Evident difference in wind speed in Weather Source and Darksky

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','windSpd100mAvg','windSpd100mMax','windSpd100mMin','windSpd80mAvg', 'windSpd80mMax', 'windSpd80mMin', 'windSpdAvg', 'windSpdMax', 'windSpdMin', 'windSpeed']], ['Date']))
    new_labels = ["Weather_Source_windSpd100mAvg","Weather_Source_windSpd100mMax","Weather_Source_windSpd100mMin","Weather_Source_windSpd80mAvg", "Weather_Source_windSpd80mMax", "Weather_Source_windSpd80mMin",
                  'Weather_Source_windSpdAvg', 'Weather_Source_windSpdMax', 'Weather_Source_windSpdMin', 'Darksky_windSpeed']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Speed in Miles per hour',title="Wind Speed Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

In [ ]:
wind_dir = merged_data.filter(regex=("windD"))
print(list(wind_dir.columns))
wind_breaing = merged_data.filter(regex=("windB"))
print(list(wind_breaing.columns))

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','windDir100mAvg','windDir80mAvg','windDirAvg','windBearing']], ['Date']))
    new_labels = ['Weather_Source_windDir100mAvg', 'Weather_Source_windDir80mAvg', 'Weather_Source_windDirAvg', 'Darksky_windBearing']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Direction in Degrees',title="Wind Direction Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Evident difference in Weather Source wind direction and Dark sky wind direction

In [ ]:
temp = merged_data.filter(regex=("temp"))
print(list(temp.columns))

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','tempMax', 'temperatureHigh', 'temperatureMax']], ['Date']))
    new_labels = ['Weather_Source_tempMax', 'Darksky_temperatureHigh','Darksky_temperatureMax']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Temperature in Fahrenheit',title="Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Maximum temperature difference between Weather Source and Dark-sky

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','tempMin', 'temperatureLow', 'temperatureMin']], ['Date']))
    new_labels = ['Weather_Source_tempMin', 'Darksky_temperatureLow','Darksky_temperatureMin']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Temperature in Fahrenheit',title="Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','tempAvg','tempMax','tempMin','temperatureHigh', 'temperatureLow', 'temperatureMax', 'temperatureMin']], ['Date']))
    new_labels = ['Weather_Source_tempAvg', 'Weather_Source_tempMax', 'Weather_Source_tempMin', 'Darksky_temperatureHigh', 'Darksky_temperatureLow', 'Darksky_temperatureMax', 'Darksky_temperatureMin']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Temperature in Fahrenheit',title="Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Huge difference in Weather Source Temp Max and Darksky temperature Max

In [ ]:
cloudcover = merged_data.filter(regex=("cl"))
print(list(cloudcover.columns))

In [ ]:
# bring cloud-cover to equal scale
merged_data['cloudCover'] = merged_data['cloudCover']*100

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','cldCvrAvg','cldCvrMax','cldCvrMin','cloudCover']], ['Date']))
    new_labels = ["Weather_Source_cldCvrAvg","Weather_Source_cldCvrMax","Weather_Source_cldCvrMin","Darksky_cloudCover"]
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Cloud Cover in %',title="CloudCover Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Weather Attributes',labels=new_labels)

Difference in Weather Source cloudcover avg and Dark sky cloud cover

In [ ]:
feels_like = merged_data.filter(regex=("feel"))
print(list(feels_like.columns))
apparent = merged_data.filter(regex=("apparent"))
print(list(apparent.columns))

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','feelsLikeAvg','feelsLikeMax','feelsLikeMin','apparentTemperatureHigh','apparentTemperatureLow', 'apparentTemperatureMax', 'apparentTemperatureMin']], ['Date']))
    new_labels = ["Weather_Source_feelsLikeAvg","Weather_Source_feelsLikeMax","Weather_Source_feelsLikeMax","Darksky_apparentTemperatureHigh","Darksky_apparentTemperatureLow", "Darksky_apparentTemperatureMax", "Darksky_apparentTemperatureMin"]
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Feels Like Temperature in F',title="Feels Like Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Huge difference in 2nd of June Weather Source in feelsLike Max and Dark sky apparent Temperature max

In [ ]:
dew_point = merged_data.filter(regex=("dew"))
print(list(dew_point.columns))

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','dewPtAvg', 'dewPtMax', 'dewPtMin', 'dewPoint']], ['Date']))
    new_labels = ["Weather_Source_dewPtAvg","Weather_Source_dewPtMax","Weather_Source_dewPtMin","Darksky_dewPoint"]
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Dew Point in F',title="Dew Point Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Weather Source Dewpoint and Darksky Dewpoint is very different

In [ ]:
msl_pres = merged_data.filter(regex=("msl"))
print(list(msl_pres.columns))
sfc_pres = merged_data.filter(regex=("sfcPres"))
print(list(sfc_pres.columns))
pres = merged_data.filter(regex=("pres"))
print(list(pres.columns))

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','sfcPresAvg', 'sfcPresMax', 'sfcPresMin', 'presTendAvg', 'presTendMax', 'presTendMin', 'mslPresAvg', 'mslPresMax', 'mslPresMin', 'pressure']], ['Date']))
    new_labels = ["Weather_Source_sfcPresAvg","Weather_Source_sfcPresMax","Weather_Source_sfcPresMin","Weather_Source_presTendAvg",'Weather_Source_presTendMax','Weather_Source_presTendMin','Weather_Source_mslPresAvg',
                  'Weather_Source_mslPresMax', 'Weather_Source_mslPresMin', "Darksky_pressure"]
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Pressure (in Millibars)',title="Pressure Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Weather Source and Dark sky Pressure are alomost similar

In [ ]:
humm = merged_data.filter(regex=("Hum"))
print(list(humm.columns))
hum = merged_data.filter(regex=("hum"))
print(list(hum.columns))

In [ ]:
merged_data['humidity'] = merged_data['humidity'] * 100

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','relHumAvg', 'relHumMax', 'relHumMin', 'humidity']], ['Date']))
    new_labels = ["Weather_Source_relHumAvg","Weather_Source_relHumMax","Weather_Source_relHumMin","Darksky_humidity"]
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Humidity (Percent (between 0 and 100))',title="Humidity Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

Weather Source average humidity and Dark-sky humidity is different 

## **Comparison of Weather Markers**

### **See if weather data is different for the 4 markers**

In [ ]:
df_ads = pd.read_csv('gs://aes-datahub-0001-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_DATASET_V8.6.csv')
df_ads = df_ads.loc[:, ~df_ads.columns.str.contains('^Unnamed')]
ads = df_ads[df_ads.EVENT=='STORM']

import folium
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[39.79539483799981,-86.14932829759584],zoom_start = 11) 

# Ensure you're handing it floats
ads['LAT'] = ads['LAT'].astype(float)
ads['LONG'] = ads['LONG'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = ads[['LAT', 'LONG']]

# Create weight column, using date
heat_df = heat_df.dropna(axis=0, subset=['LAT','LONG'])
heat_df['combined'] = list(zip(heat_df.LAT, heat_df.LONG))

heat_data = list(heat_df.combined)

folium.Marker([39.9613, -86.4034],
              popup='Marker 1 [Lat,Long]=[39.9613, -86.4034]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.8971, -86.3045],
              popup='Marker 2 [Lat,Long]=[39.8971, -86.3045]',
              icon=folium.Icon(color='green')
             ).add_to(map_hooray)

folium.Marker([39.9060, -86.2001],
              popup='Marker 3 [Lat,Long]=[39.9060, -86.2001]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.9024, -86.0738],
              popup='Marker 4 [Lat,Long]=[39.9024, -86.0738]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.8960, -85.9783],
              popup='Marker 5 [Lat,Long]=[39.8960, -85.9783]',
              icon=folium.Icon(color='green')
             ).add_to(map_hooray)

folium.Marker([39.8339, -86.3155],
              popup='Marker 6 [Lat,Long]=[39.8339, -86.3155]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.8412, -86.2056],
              popup='Marker 7 [Lat,Long]=[39.8412, -86.2056]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.8381, -86.0985],
              popup='Marker 8 [Lat,Long]=[39.8381, -86.0985]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.8386, -85.9811],
              popup='Marker 9 [Lat,Long]=[39.8386, -85.9811]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.7579, -86.3155],
              popup='Marker 10 [Lat,Long]=[39.7579, -86.3155]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.7621, -86.2042],
              popup='Marker 11 [Lat,Long]=[39.7621, -86.2042]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.7621, -86.0923],
              popup='Marker 12 [Lat,Long]=[39.7621, -86.0923]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.7695, -85.9708],
              popup='Marker 13 [Lat,Long]=[39.7695, -85.9708]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.6617, -86.2935],
              popup='Marker 14 [Lat,Long]=[39.6617, -86.2935]',
              icon=folium.Icon(color='green')
             ).add_to(map_hooray)

folium.Marker([39.6639, -86.1823],
              popup='Marker 15 [Lat,Long]=[39.6639, -86.1823]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.6702, -86.0669],
              popup='Marker 16 [Lat,Long]=[39.6702, -86.0669]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.6744, -85.9557],
              popup='Marker 17 [Lat,Long]=[39.6744, -85.9557]',
              icon=folium.Icon(color='green')
             ).add_to(map_hooray)

folium.Marker([39.5909, -86.4212],
              popup='Marker 18 [Lat,Long]=[39.5909, -86.4212]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)
 
folium.Marker([39.5295, -86.5874],
              popup='Marker 19 [Lat,Long]=[39.5295, -86.5874]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Marker([39.5475, -86.2743],
              popup='Marker 20 [Lat,Long]=[39.5475, -86.2743]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)


# Display the map
map_hooray

In [ ]:
marker_dict = {
    'Marker 2' :['39.8971', '-86.3045'],
    'Marker 5' :['39.8960', '-85.9783'],
    'Marker 14':['39.6617', '-86.2935'],
    'Marker 17':['39.6744', '-85.9557']
}


start_date = "2020-06-01"
end_date = "2020-06-20"

weathersource_marker2 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('Marker 2')[0], long=marker_dict.get('Marker 2')[0], period='day')
weathersource_marker5 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('Marker 5')[0], long=marker_dict.get('Marker 5')[0], period='day')
weathersource_marker14 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('Marker 14')[0], long=marker_dict.get('Marker 14')[0], period='day')
weathersource_marker17 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('Marker 17')[0], long=marker_dict.get('Marker 17')[0], period='day')

In [ ]:
weathersource_marker2['date_time'] = pd.to_datetime(weathersource_marker2['timestamp'])
weathersource_marker2['Date'] = weathersource_marker2['date_time'].dt.date

weathersource_marker5['date_time'] = pd.to_datetime(weathersource_marker5['timestamp'])
weathersource_marker5['Date'] = weathersource_marker5['date_time'].dt.date

weathersource_marker14['date_time'] = pd.to_datetime(weathersource_marker14['timestamp'])
weathersource_marker14['Date'] = weathersource_marker14['date_time'].dt.date

weathersource_marker17['date_time'] = pd.to_datetime(weathersource_marker17['timestamp'])
weathersource_marker17['Date'] = weathersource_marker17['date_time'].dt.date

In [ ]:
weathersource_marker2.drop(['date_time'],axis=1,inplace=True)
weathersource_marker5.drop(['date_time'],axis=1,inplace=True)
weathersource_marker14.drop(['date_time'],axis=1,inplace=True)
weathersource_marker17.drop(['date_time'],axis=1,inplace=True)

In [ ]:
marker2_cols = weathersource_marker2.columns[~weathersource_marker2.columns.isin(['Date'])]
weathersource_marker2.rename(columns = dict(zip(marker2_cols, 'Marker2_' + marker2_cols)), inplace=True)

marker5_cols = weathersource_marker5.columns[~weathersource_marker5.columns.isin(['Date'])]
weathersource_marker5.rename(columns = dict(zip(marker5_cols, 'Marker5_' + marker5_cols)), inplace=True)

marker14_cols = weathersource_marker14.columns[~weathersource_marker14.columns.isin(['Date'])]
weathersource_marker14.rename(columns = dict(zip(marker14_cols, 'Marker14_' + marker14_cols)), inplace=True)

marker17_cols = weathersource_marker17.columns[~weathersource_marker17.columns.isin(['Date'])]
weathersource_marker17.rename(columns = dict(zip(marker17_cols, 'Marker17_' + marker17_cols)), inplace=True)

In [ ]:
m1 = pd.merge(weathersource_marker2,weathersource_marker5, on=['Date'], how='left')
m4 = pd.merge(m1,weathersource_marker14, on=['Date'], how='left')
merged_data = pd.merge(m4,weathersource_marker17, on=['Date'], how='left')

In [ ]:
data = merged_data[['Date','Marker2_windSpdAvg', 'Marker5_windSpdAvg', 'Marker14_windSpdAvg', 'Marker17_windSpdAvg', 
                    'Marker2_windDirAvg', 'Marker5_windDirAvg', 'Marker14_windDirAvg', 'Marker17_windDirAvg',
                    'Marker2_tempAvg', 'Marker5_tempAvg', 'Marker14_tempAvg', 'Marker17_tempAvg',
                    'Marker2_cldCvrAvg', 'Marker5_cldCvrAvg', 'Marker14_cldCvrAvg', 'Marker17_cldCvrAvg', 
                    'Marker2_feelsLikeAvg', 'Marker5_feelsLikeAvg', 'Marker14_feelsLikeAvg', 'Marker17_feelsLikeAvg', 
                    'Marker2_dewPtAvg', 'Marker5_dewPtAvg', 'Marker14_dewPtAvg', 'Marker17_dewPtAvg',
                    'Marker2_sfcPresAvg', 'Marker5_sfcPresAvg', 'Marker14_sfcPresAvg', 'Marker17_sfcPresAvg',
                    'Marker2_relHumAvg', 'Marker2_relHumAvg', 'Marker2_relHumAvg', 'Marker2_relHumAvg'
                    ]]
data

### **Comparison of Wind Speed between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date', 'Marker2_windSpdAvg','Marker5_windSpdAvg','Marker14_windSpdAvg','Marker17_windSpdAvg']], ['Date']))
    new_labels = ['Marker2_windSpdAvg','Marker5_windSpdAvg','Marker14_windSpdAvg','Marker17_windSpdAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Speed in Miles per hour',title="Wind Speed Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### **Comparison of Wind Direction between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','Marker2_windDirAvg','Marker5_windDirAvg','Marker14_windDirAvg','Marker17_windDirAvg']], ['Date']))
    new_labels = ['Marker2_windDirAvg','Marker5_windDirAvg','Marker14_windDirAvg','Marker17_windDirAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Direction in Degrees',title="Wind Direction Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### **Comparison of Temperature between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date', 'Marker2_tempAvg','Marker5_tempAvg', 'Marker14_tempAvg','Marker17_tempAvg']], ['Date']))
    new_labels = ['Marker2_tempAvg','Marker5_tempAvg', 'Marker14_tempAvg','Marker17_tempAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Temperature in Fahrenheit',title="Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### **Comparison of CloudCover between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','Marker2_cldCvrAvg', 'Marker5_cldCvrAvg','Marker14_cldCvrAvg', 'Marker17_cldCvrAvg']], ['Date']))
    new_labels = ['Marker2_cldCvrAvg', 'Marker5_cldCvrAvg','Marker14_cldCvrAvg', 'Marker17_cldCvrAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Cloud Cover in %',title="CloudCover Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Weather Attributes',labels=new_labels)

### **Comparison of Feels like Temperature between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','Marker2_feelsLikeAvg','Marker5_feelsLikeAvg','Marker14_feelsLikeAvg','Marker17_feelsLikeAvg']], ['Date']))
    new_labels = ['Marker2_feelsLikeAvg','Marker5_feelsLikeAvg','Marker14_feelsLikeAvg','Marker17_feelsLikeAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Feels Like Temperature in F',title="Feels Like Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### **Comparison of Dew Point between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','Marker2_dewPtAvg','Marker5_dewPtAvg','Marker14_dewPtAvg', 'Marker17_dewPtAvg']], ['Date']))
    new_labels = ['Marker2_dewPtAvg','Marker5_dewPtAvg','Marker14_dewPtAvg', 'Marker17_dewPtAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Dew Point in F',title="Dew Point Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### **Comparison of Pressure between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','Marker2_sfcPresAvg', 'Marker5_sfcPresAvg', 'Marker14_sfcPresAvg', 'Marker17_sfcPresAvg']], ['Date']))
    new_labels = ['Marker2_sfcPresAvg', 'Marker5_sfcPresAvg', 'Marker14_sfcPresAvg', 'Marker17_sfcPresAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Pressure (in Millibars)',title="Pressure Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### **Comparison of Humidity between the 4 location Markers**

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','Marker2_relHumAvg','Marker5_relHumAvg','Marker14_relHumAvg','Marker17_relHumAvg']], ['Date']))
    new_labels = ['Marker2_relHumAvg','Marker5_relHumAvg','Marker14_relHumAvg','Marker17_relHumAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Humidity (Percent (between 0 and 100))',title="Humidity Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

## **Weather Sensitivity for 1 mile and 5 mile**

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[39.79539483799981,-86.14932829759584],zoom_start = 12) 

folium.Marker([39.7621, -86.0923],
              popup='Marker 12 [Lat,Long]=[39.7621, -86.0923]',
              icon=folium.Icon(color='orange')
             ).add_to(map_hooray)

folium.Circle([39.7621, -86.0923],radius=1609.34,color='#808080').add_to(map_hooray)
folium.Circle([39.7621, -86.0923],radius=8046.72,color='#808080').add_to(map_hooray)

folium.Marker([39.7645, -85.9986],popup='5miles [Lat,Long]=[39.7645, -85.9986]',icon=folium.Icon(color='gray')).add_to(map_hooray)
folium.Marker([39.7550, -86.1860],popup='5miles [Lat,Long]=[39.7550, -86.1860]',icon=folium.Icon(color='gray')).add_to(map_hooray)
folium.Marker([39.7766, -86.0937],popup='1mile [Lat,Long]=[39.7766, -86.0937]',icon=folium.Icon(color='lightgray')).add_to(map_hooray)
folium.Marker([39.7484, -86.0896],popup='1mile [Lat,Long]=[39.7484, -86.0896]',icon=folium.Icon(color='lightgray')).add_to(map_hooray)


# Display the map
map_hooray

In [ ]:
marker_dict = {
    '5miles_location1' :['39.7645', '-85.9986'],
    '5miles_location2' :['39.7550', '-86.1860'],
    '1mile_location3':['39.7766', '-86.0937'],
    '1mile_location4':['39.7484', '-86.0896']
}


start_date = "2020-06-01"
end_date = "2020-06-20"

weathersource_marker2 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('5miles_location1')[0], long=marker_dict.get('5miles_location1')[1], period='day')
weathersource_marker5 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('5miles_location2')[0], long=marker_dict.get('5miles_location2')[1], period='day')
weathersource_marker14 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('1mile_location3')[0], long=marker_dict.get('1mile_location3')[1], period='day')
weathersource_marker17 = ws_historical_data(start=start_date, end=end_date, lat=marker_dict.get('1mile_location4')[0], long=marker_dict.get('1mile_location4')[1], period='day')

weathersource_marker2['date_time'] = pd.to_datetime(weathersource_marker2['timestamp'])
weathersource_marker2['Date'] = weathersource_marker2['date_time'].dt.date

weathersource_marker5['date_time'] = pd.to_datetime(weathersource_marker5['timestamp'])
weathersource_marker5['Date'] = weathersource_marker5['date_time'].dt.date

weathersource_marker14['date_time'] = pd.to_datetime(weathersource_marker14['timestamp'])
weathersource_marker14['Date'] = weathersource_marker14['date_time'].dt.date

weathersource_marker17['date_time'] = pd.to_datetime(weathersource_marker17['timestamp'])
weathersource_marker17['Date'] = weathersource_marker17['date_time'].dt.date

weathersource_marker2.drop(['date_time'],axis=1,inplace=True)
weathersource_marker5.drop(['date_time'],axis=1,inplace=True)
weathersource_marker14.drop(['date_time'],axis=1,inplace=True)
weathersource_marker17.drop(['date_time'],axis=1,inplace=True)

marker2_cols = weathersource_marker2.columns[~weathersource_marker2.columns.isin(['Date'])]
weathersource_marker2.rename(columns = dict(zip(marker2_cols, '5miles_location1_' + marker2_cols)), inplace=True)

marker5_cols = weathersource_marker5.columns[~weathersource_marker5.columns.isin(['Date'])]
weathersource_marker5.rename(columns = dict(zip(marker5_cols, '5miles_location2_' + marker5_cols)), inplace=True)

marker14_cols = weathersource_marker14.columns[~weathersource_marker14.columns.isin(['Date'])]
weathersource_marker14.rename(columns = dict(zip(marker14_cols, '1mile_location3_' + marker14_cols)), inplace=True)

marker17_cols = weathersource_marker17.columns[~weathersource_marker17.columns.isin(['Date'])]
weathersource_marker17.rename(columns = dict(zip(marker17_cols, '1mile_location4_' + marker17_cols)), inplace=True)

m1 = pd.merge(weathersource_marker2,weathersource_marker5, on=['Date'], how='left')
m4 = pd.merge(m1,weathersource_marker14, on=['Date'], how='left')
merged_data = pd.merge(m4,weathersource_marker17, on=['Date'], how='left')
# merged_data.head()

In [ ]:
data = merged_data[['Date','5miles_location1_windSpdAvg', '5miles_location2_windSpdAvg', '1mile_location3_windSpdAvg', '1mile_location4_windSpdAvg', 
                    '5miles_location1_windDirAvg', '5miles_location2_windDirAvg', '1mile_location3_windDirAvg', '1mile_location4_windDirAvg',
                    '5miles_location1_tempAvg', '5miles_location2_tempAvg', '1mile_location3_tempAvg', '1mile_location4_tempAvg',
                    '5miles_location1_cldCvrAvg', '5miles_location2_cldCvrAvg', '1mile_location3_cldCvrAvg', '1mile_location4_cldCvrAvg', 
                    '5miles_location1_feelsLikeAvg', '5miles_location2_feelsLikeAvg', '1mile_location3_feelsLikeAvg', '1mile_location4_feelsLikeAvg', 
                    '5miles_location1_dewPtAvg', '5miles_location2_dewPtAvg', '1mile_location3_dewPtAvg', '1mile_location4_dewPtAvg',
                    '5miles_location1_sfcPresAvg', '5miles_location2_sfcPresAvg', '1mile_location3_sfcPresAvg', '1mile_location4_sfcPresAvg',
                    '5miles_location1_relHumAvg', '5miles_location2_relHumAvg', '1mile_location3_relHumAvg', '1mile_location4_relHumAvg'
                    ]]
data

### Comparison of Wind Speed between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date', '5miles_location1_windSpdAvg','5miles_location2_windSpdAvg','1mile_location3_windSpdAvg','1mile_location4_windSpdAvg']], ['Date']))
    new_labels = ['5miles_location1_windSpdAvg','5miles_location2_windSpdAvg','1mile_location3_windSpdAvg','1mile_location4_windSpdAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Speed in Miles per hour',title="Wind Speed Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### Comparison of Wind Direction between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','5miles_location1_windDirAvg','5miles_location2_windDirAvg','1mile_location3_windDirAvg','1mile_location4_windDirAvg']], ['Date']))
    new_labels = ['5miles_location1_windDirAvg','5miles_location2_windDirAvg','1mile_location3_windDirAvg','1mile_location4_windDirAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Wind Direction in Degrees',title="Wind Direction Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### Comparison of Temperature between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date', '5miles_location1_tempAvg','5miles_location2_tempAvg', '1mile_location3_tempAvg','1mile_location4_tempAvg']], ['Date']))
    new_labels = ['5miles_location1_tempAvg','5miles_location2_tempAvg', '1mile_location3_tempAvg','1mile_location4_tempAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Temperature in Fahrenheit',title="Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### Comparison of CloudCover between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                      data=pd.melt(merged_data[['Date','5miles_location1_cldCvrAvg', '5miles_location2_cldCvrAvg','1mile_location3_cldCvrAvg', '1mile_location4_cldCvrAvg']], ['Date']))
    new_labels = ['5miles_location1_cldCvrAvg', '5miles_location2_cldCvrAvg','1mile_location3_cldCvrAvg', '1mile_location4_cldCvrAvg']
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Cloud Cover in %',title="CloudCover Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Weather Attributes',labels=new_labels)

### Comparison of Feels Like Temperature between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','5miles_location1_feelsLikeAvg','5miles_location2_feelsLikeAvg','1mile_location3_feelsLikeAvg','1mile_location4_feelsLikeAvg']], ['Date']))
    new_labels = ['5miles_location1_feelsLikeAvg','5miles_location2_feelsLikeAvg','1mile_location3_feelsLikeAvg','1mile_location4_feelsLikeAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Feels Like Temperature in F',title="Feels Like Temperature Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### Comparison of Dewpoint between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','5miles_location1_dewPtAvg','5miles_location2_dewPtAvg','1mile_location3_dewPtAvg', '1mile_location4_dewPtAvg']], ['Date']))
    new_labels = ['5miles_location1_dewPtAvg','5miles_location2_dewPtAvg','1mile_location3_dewPtAvg', '1mile_location4_dewPtAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Dew Point in F',title="Dew Point Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### Comparison of Pressure between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','5miles_location1_sfcPresAvg', '5miles_location2_sfcPresAvg', '1mile_location3_sfcPresAvg', '1mile_location4_sfcPresAvg']], ['Date']))
    new_labels = ['5miles_location1_sfcPresAvg', '5miles_location2_sfcPresAvg', '1mile_location3_sfcPresAvg', '1mile_location4_sfcPresAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Pressure (in Millibars)',title="Pressure Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')

### Comparison of Humidity between 1 mile and 5 mile distance

In [ ]:
sns.set(rc={'figure.figsize':(26,10)})
with plt.rc_context({'lines.linewidth': 2.5}):
    ax = sns.lineplot(x='Date', y='value', hue='variable', 
                  data=pd.melt(merged_data[['Date','5miles_location1_relHumAvg','5miles_location2_relHumAvg','1mile_location3_relHumAvg','1mile_location4_relHumAvg']], ['Date']))
    new_labels = ['5miles_location1_relHumAvg','5miles_location2_relHumAvg','1mile_location3_relHumAvg','1mile_location4_relHumAvg']
    
sns.set(rc={"axes.titlesize":20,'figure.facecolor':'white','axes.facecolor':'white'},style="white")
ax.set(xlabel='Date', ylabel='Humidity (Percent (between 0 and 100))',title="Humidity Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=new_labels, title='Weather Attributes')